<a href="https://colab.research.google.com/github/wannasmile/colab_code_note/blob/main/IRC018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lightgbm
!pip install gbnet

In [2]:
import time  # 导入时间模块

import lightgbm as lgb  # 导入lightgbm库
import numpy as np  # 导入numpy库
import xgboost as xgb  # 导入xgboost库
import torch  # 导入pytorch库

from gbnet import lgbmodule, xgbmodule  # 从gbnet库导入lgbmodule和xgbmodule模块

# 生成数据集
np.random.seed(100)  # 设置随机数种子
n = 1000  # 设置样本数量
input_dim = 20  # 设置输入特征维度
output_dim = 1  # 设置输出维度
X = np.random.random([n, input_dim])  # 生成随机输入特征
B = np.random.random([input_dim, output_dim])  # 生成随机权重
Y = X.dot(B) + np.random.random([n, output_dim])  # 生成目标变量

iters = 100  # 设置迭代次数
t0 = time.time()  # 记录开始时间

# 使用XGBoost进行训练以进行比较
xbst = xgb.train(
    params={'objective': 'reg:squarederror', 'base_score': 0.0},  # 设置参数
    dtrain=xgb.DMatrix(X, label=Y),  # 创建DMatrix数据集
    num_boost_round=iters  # 设置迭代次数
)
t1 = time.time()  # 记录XGBoost训练结束时间

# 使用LightGBM进行训练以进行比较
lbst = lgb.train(
    params={'verbose': -1},  # 设置参数
    train_set=lgb.Dataset(X, label=Y.flatten(), init_score=[0 for i in range(n)]),  # 创建Dataset数据集
    num_boost_round=iters  # 设置迭代次数
)
t2 = time.time()  # 记录LightGBM训练结束时间

# 使用XGBModule进行训练
xnet = xgbmodule.XGBModule(n, input_dim, output_dim, params={})  # 初始化XGBModule模型
xmse = torch.nn.MSELoss()  # 定义均方误差损失函数

X_dmatrix = xgb.DMatrix(X)  # 创建DMatrix数据集
for i in range(iters):  # 循环迭代
    xnet.zero_grad()  # 清空梯度
    xpred = xnet(X_dmatrix)  # 进行预测

    loss = 1 / 2 * xmse(xpred, torch.Tensor(Y))  # 计算损失
    loss.backward(create_graph=True)  # 反向传播计算梯度

    xnet.gb_step()  # 更新模型参数
xnet.eval()  # 切换到评估模式
t3 = time.time()  # 记录XGBModule训练结束时间

# 使用LGBModule进行训练
lnet = lgbmodule.LGBModule(n, input_dim, output_dim, params={})  # 初始化LGBModule模型
lmse = torch.nn.MSELoss()  # 定义均方误差损失函数

X_dataset = lgb.Dataset(X)  # 创建Dataset数据集
for i in range(iters):  # 循环迭代
    lnet.zero_grad()  # 清空梯度
    lpred = lnet(X_dataset)  # 进行预测

    loss = lmse(lpred, torch.Tensor(Y))  # 计算损失
    loss.backward(create_graph=True)  # 反向传播计算梯度

    lnet.gb_step()  # 更新模型参数
lnet.eval()  # 切换到评估模式
t4 = time.time()  # 记录LGBModule训练结束时间

print(np.max(np.abs(xbst.predict(xgb.DMatrix(X)) - xnet(X_dmatrix).detach().numpy().flatten())))  # 打印XGBoost和XGBModule预测结果的差异
print(np.max(np.abs(lbst.predict(X) - lnet(X).detach().numpy().flatten())))  # 打印LightGBM和LGBModule预测结果的差异
print(f'xgboost time: {t1 - t0}')  # 打印XGBoost训练时间
print(f'lightgbm time: {t2 - t1}')  # 打印LightGBM训练时间
print(f'xgbmodule time: {t3 - t2}')  # 打印XGBModule训练时间
print(f'lgbmodule time: {t4 - t3}')  # 打印LGBModule训练时间

/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: Using backward() with create_graph=True will create a reference cycle between the parameter and its gradient which can cause a memory leak. We recommend using autograd.grad when creating the graph to avoid this. If you have to use this function, make sure to reset the .grad fields of your parameters to None after use to break the cycle and avoid the leak. (Triggered internally at /pytorch/torch/csrc/autograd/engine.cpp:1260.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


1.9073486e-06
2.6987259360566895e-07
xgboost time: 0.8300571441650391
lightgbm time: 0.21436691284179688
xgbmodule time: 0.9426262378692627
lgbmodule time: 0.3606119155883789


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

# 1. 配置类
class Config:
    """
    配置类，用于存储模型和数据相关的参数。
    """
    def __init__(self):
        self.feature_dim = 100  # 输入特征维度
        self.num_experts = 2  # Expert 数量 (新老用户)
        self.hidden_size = 128  # 隐藏层大小
        self.num_layers = 3 # 网络层数
        self.dropout_rate = 0.2  # dropout 概率
        self.learning_rate = 0.001  # 学习率
        self.batch_size = 256  # 批量大小
        self.num_epochs = 50  # 训练 epoch 数量
        self.test_size = 0.2  # 测试集比例
        self.distill_weight = 0.1 #  知识蒸馏损失权重
        self.consistency_weight = 0.1 # 一致性损失权重
        self.teacher_prob_dim = 4 # teacher model 输出概率维度
        self.use_teacher_probs = True # 是否使用教师模型概率
        self.focal_loss_gamma = 2.0 # Focal Loss 的 gamma 参数
        self.temperature = 3.0 # 知识蒸馏温度参数
        self.use_shadow_samples = False # 是否使用影子样本
        self.use_time_series = False # 是否使用时间序列特征

cfg = Config()

# 2. 模型定义
class GatedDenseBlock(nn.Module):
    """
    门控 Dense Block，包含一个全连接层和一个门控机制。
    """
    def __init__(self, input_size, output_size):
        super(GatedDenseBlock, self).__init__()
        self.dense = nn.Linear(input_size, output_size) # 定义一个线性层 (全连接层)，输入大小为 input_size，输出大小为 output_size
        self.gate = nn.Sequential( # 定义一个序列化的模块，包含一个线性层和一个 Sigmoid 激活函数
            nn.Linear(input_size, output_size), # 线性层，输入大小为 input_size，输出大小为 output_size
            nn.Sigmoid() # Sigmoid 激活函数，将输出值映射到 0 到 1 之间
        )

    def forward(self, x):
        """
        前向传播函数。
        Args:
            x (torch.Tensor): 输入张量，形状为 (batch_size, input_size).
        Returns:
            torch.Tensor: 输出张量，形状为 (batch_size, output_size).
        """
        output = self.dense(x) # 将输入 x 通过线性层 dense，得到输出 output
        gate_value = self.gate(x) # 将输入 x 通过门控模块 gate，得到门控值 gate_value
        return output * gate_value # 返回输出 output 和门控值 gate_value 的乘积，实现门控机制

class UnifiedRiskModel(nn.Module):
    """
    统一风险模型，包含新老用户区分网络和专家网络。
    """
    def __init__(self, cfg):
        super(UnifiedRiskModel, self).__init__()
        self.cfg = cfg
        self.num_experts = cfg.num_experts # 2，表示新老用户
        self.feature_dim = cfg.feature_dim
        self.hidden_size = cfg.hidden_size
        self.num_layers = cfg.num_layers
        self.dropout_rate = cfg.dropout_rate
        self.teacher_prob_dim = cfg.teacher_prob_dim

        # 新老用户区分网络 (Bi-Gated Dense Block)
        self.bi_gated_dense = nn.Sequential(
            GatedDenseBlock(self.feature_dim + self.teacher_prob_dim, self.hidden_size), # 输入包含原始特征和教师模型概率
            nn.Dropout(self.dropout_rate),
            GatedDenseBlock(self.hidden_size, self.hidden_size),
            nn.Dropout(self.dropout_rate)
        )

        # 新用户专家通道
        self.new_user_suspect_net = nn.Sequential( # 预测嫌疑
            GatedDenseBlock(self.hidden_size + self.teacher_prob_dim, self.hidden_size), # 输入包含共享层输出和教师模型概率
            nn.Dropout(self.dropout_rate),
            nn.Linear(self.hidden_size, 1),
        )
        self.new_user_chargeback_net = nn.Sequential( # 预测拒付
            GatedDenseBlock(self.hidden_size + self.teacher_prob_dim, self.hidden_size),
            nn.Dropout(self.dropout_rate),
            nn.Linear(self.hidden_size, 1),
        )

        # 老用户专家通道
        self.old_user_suspect_net = nn.Sequential( # 预测嫌疑
            GatedDenseBlock(self.hidden_size + self.teacher_prob_dim, self.hidden_size),
            nn.Dropout(self.dropout_rate),
            nn.Linear(self.hidden_size, 1),
        )
        self.old_user_chargeback_net = nn.Sequential( # 预测拒付
            GatedDenseBlock(self.hidden_size + self.teacher_prob_dim, self.hidden_size),
            nn.Dropout(self.dropout_rate),
            nn.Linear(self.hidden_size, 1),
        )

        # 注意力融合层
        self.attention_layer = nn.Sequential(
            nn.Linear(4, 2),  # 输入维度为4 (新/老用户嫌疑和拒付预测共4个)，输出注意力权重
            nn.Softmax(dim=1)
        )

        # 最终拒付预测层
        self.final_chargeback_net = nn.Sequential(
            nn.Linear(2, self.hidden_size), # 输入融合的新老用户特征
            nn.Dropout(self.dropout_rate),
            nn.Linear(self.hidden_size, 1),
        )

        self.sigmoid = nn.Sigmoid()

    def forward(self, x, user_type):
        """
        前向传播函数。
        Args:
            x (torch.Tensor): 输入特征张量，形状为 (batch_size, feature_dim + teacher_prob_dim + 1).
            user_type (torch.Tensor): 用户类型张量，形状为 (batch_size,).
        Returns:
            tuple: 包含新老用户预测概率的元组，形状都为 (batch_size, 1).
        """
        batch_size = x.size(0)
        # 创建 one-hot 编码的用户类型表示
        user_type_onehot = F.one_hot(user_type.long(), num_classes=self.num_experts).float()  # (batch_size, num_experts)

        # 提取原始特征和教师模型概率
        original_features = x[:, :self.feature_dim] # (batch_size, feature_dim)
        teacher_probs = x[:, self.feature_dim+1:self.feature_dim + self.teacher_prob_dim + 1] # (batch_size, teacher_prob_dim)

        # 共享特征提取
        combined_features = torch.cat([original_features, teacher_probs], dim=1)
        shared_features = self.bi_gated_dense(combined_features) # (batch_size, hidden_size)

        # 新用户专家通道
        new_user_features = torch.cat([shared_features, teacher_probs], dim=1) # 拼接共享特征和教师模型概率
        new_user_suspect_prob = self.sigmoid(self.new_user_suspect_net(new_user_features)) # (batch_size, 1)
        new_user_chargeback_prob = self.sigmoid(self.new_user_chargeback_net(new_user_features)) # (batch_size, 1)

        # 老用户专家通道
        old_user_features = torch.cat([shared_features, teacher_probs], dim=1)
        old_user_suspect_prob = self.sigmoid(self.old_user_suspect_net(old_user_features)) # (batch_size, 1)
        old_user_chargeback_prob = self.sigmoid(self.old_user_chargeback_net(old_user_features)) # (batch_size, 1)

        # 注意力融合
        attention_input = torch.cat([new_user_suspect_prob, new_user_chargeback_prob, old_user_suspect_prob, old_user_chargeback_prob], dim=1) # (batch_size, 4)
        attention_weights = self.attention_layer(attention_input) # (batch_size, 2)

        #  将新老用户的输出拼接
        new_user_output = torch.cat([new_user_suspect_prob, new_user_chargeback_prob], dim=1)
        old_user_output = torch.cat([old_user_suspect_prob, old_user_chargeback_prob], dim=1)
        fused_features = attention_weights[:, 0].unsqueeze(1) * new_user_output + attention_weights[:, 1].unsqueeze(1) * old_user_output # (batch_size, 2)


        # 最终拒付预测
        final_chargeback_prob = self.sigmoid(self.final_chargeback_net(fused_features)) # (batch_size, 1)

        return new_user_suspect_prob, new_user_chargeback_prob, old_user_suspect_prob, old_user_chargeback_prob, final_chargeback_prob, attention_weights

# 3. 损失函数
def compute_distillation_loss(student_logits, teacher_logits, temperature):
    """
    计算知识蒸馏损失。
    Args:
        student_logits (torch.Tensor): 学生模型 logits.
        teacher_logits (torch.Tensor): 教师模型 logits.
        temperature (float): 温度参数.
    Returns:
        torch.Tensor: 蒸馏损失.
    """
    p_s = F.log_softmax(student_logits / temperature, dim=-1)
    p_t = F.softmax(teacher_logits / temperature, dim=-1)
    loss = F.kl_div(p_s, p_t, reduction='batchmean') * (temperature ** 2)
    return loss

def compute_consistency_loss(suspect_prob_new, chargeback_prob):
    """
    计算新老用户预测一致性损失。这里改为计算新老用户各自的嫌疑预测和拒付预测之间的一致性。

    Args:
        suspect_prob_new (torch.Tensor): 新用户疑似欺诈概率.
        chargeback_prob (torch.Tensor): 新用户拒付概率.

    Returns:
        torch.Tensor: 一致性损失.
    """
    loss = F.mse_loss(suspect_prob_new, chargeback_prob)
    return loss

def focal_loss(logits, labels, gamma=2.0, reduction='mean'):
    """
    计算 Focal Loss，用于处理类别不平衡问题。
    Args:
        logits (torch.Tensor): 模型的预测 logits.
        labels (torch.Tensor): 真实的标签.
        gamma (float):  聚焦参数，gamma越大，越关注难分类样本.
        reduction (str): 'none' | 'mean' | 'sum': Specifies the reduction to apply to the output
    Returns:
        torch.Tensor: 损失值.
    """
    ce_loss = F.binary_cross_entropy_with_logits(logits, labels, reduction='none')
    p_t = torch.exp(-ce_loss)
    focal_loss = (1 - p_t) ** gamma * ce_loss
    if reduction == 'mean':
        return torch.mean(focal_loss)
    elif reduction == 'sum':
        return torch.sum(focal_loss)
    else:
        return focal_loss

def compute_loss(cfg, new_user_suspect_prob, new_user_chargeback_prob, old_user_suspect_prob, old_user_chargeback_prob, final_chargeback_prob, y, user_type, teacher_probs=None):
    """
    计算总损失，包括疑似欺诈损失、chargeback 损失、知识蒸馏损失和一致性损失。
    Args:
        cfg (Config): 配置对象.
        new_user_suspect_prob (torch.Tensor): 新用户疑似欺诈概率.
        new_user_chargeback_prob (torch.Tensor): 新用户 chargeback 概率.
        old_user_suspect_prob (torch.Tensor): 老用户疑似欺诈概率.
        old_user_chargeback_prob (torch.Tensor): 老用户 chargeback 概率.
        final_chargeback_prob (torch.Tensor): 最终 chargeback 概率.
        y (torch.Tensor): 标签，形状为 (batch_size,).
        user_type (torch.Tensor): 用户类型，形状为 (batch_size,).
        teacher_probs (dict, optional): 包含四个教师模型预测概率的字典.
            key 为模型名称 (A, B, C, D)，value 为对应的预测概率 DataFrame。
            默认为 None，表示不使用教师模型概率。
    Returns:
        tuple: 包含总损失和各项损失的元组.
    """
    # 疑似欺诈损失 (新老用户)
    L_suspect_new = F.binary_cross_entropy(new_user_suspect_prob, y.unsqueeze(1))
    L_suspect_old = focal_loss(old_user_suspect_prob, y.unsqueeze(1), gamma=cfg.focal_loss_gamma) # 老用户使用Focal Loss
    L_suspect = L_suspect_new + L_suspect_old

    # chargeback 损失 (新老用户)
    L_chargeback = focal_loss(final_chargeback_prob, y.unsqueeze(1), gamma=cfg.focal_loss_gamma) # 全部使用Focal Loss

    # 知识蒸馏损失
    if teacher_probs is not None and cfg.use_teacher_probs:
        distill_losses = []
        for i in range(len(user_type)):
            user_type_val = user_type[i].item()
            if user_type_val == 0:  # 新用户
                #teacher_prob_A = teacher_probs['A']['prob_A'].iloc[i]
                #teacher_prob_B = teacher_probs['B']['prob_B'].iloc[i]
                teacher_prob_A = torch.tensor(teacher_probs['A']['prob_A'].iloc[i], dtype=torch.float32)
                teacher_prob_B = torch.tensor(teacher_probs['B']['prob_B'].iloc[i], dtype=torch.float32)
                student_logits_suspect = torch.stack([1 - new_user_suspect_prob[i], new_user_suspect_prob[i]]).squeeze()
                student_logits_chargeback = torch.stack([1 - new_user_chargeback_prob[i], new_user_chargeback_prob[i]]).squeeze()

                teacher_logits_A = torch.stack([1 - teacher_prob_A, teacher_prob_A]).squeeze()
                teacher_logits_B = torch.stack([1 - teacher_prob_B, teacher_prob_B]).squeeze()
                distill_loss_A = compute_distillation_loss(student_logits_suspect, teacher_logits_A, cfg.temperature)
                distill_loss_B = compute_distillation_loss(student_logits_chargeback, teacher_logits_B, cfg.temperature)
                distill_losses.extend([distill_loss_A, distill_loss_B])

            elif user_type_val == 1: # 老用户
                #teacher_prob_C = teacher_probs['C']['prob_C'].iloc[i]
                #teacher_prob_D = teacher_probs['D']['prob_D'].iloc[i]
                teacher_prob_C = torch.tensor(teacher_probs['C']['prob_C'].iloc[i], dtype=torch.float32)
                teacher_prob_D = torch.tensor(teacher_probs['D']['prob_D'].iloc[i], dtype=torch.float32)
                student_logits_suspect = torch.stack([1 - old_user_suspect_prob[i], old_user_suspect_prob[i]]).squeeze()
                student_logits_chargeback = torch.stack([1 - old_user_chargeback_prob[i], old_user_chargeback_prob[i]]).squeeze()
                teacher_logits_C = torch.stack([1 - teacher_prob_C, teacher_prob_C]).squeeze()
                teacher_logits_D = torch.stack([1 - teacher_prob_D, teacher_prob_D]).squeeze()
                distill_loss_C = compute_distillation_loss(student_logits_suspect, teacher_logits_C, cfg.temperature)
                distill_loss_D = compute_distillation_loss(student_logits_chargeback, teacher_logits_D, cfg.temperature)
                distill_losses.extend([distill_loss_C, distill_loss_D])
        if distill_losses:
            L_distill = torch.mean(torch.stack(distill_losses)) # (1,)
        else:
            L_distill = torch.tensor(0.0)
    else:
        L_distill = torch.tensor(0.0)

    # 一致性损失
    L_consistency_new = compute_consistency_loss(new_user_suspect_prob, new_user_chargeback_prob)
    L_consistency_old = compute_consistency_loss(old_user_suspect_prob, old_user_chargeback_prob)
    L_consistency = L_consistency_new + L_consistency_old

    # 任务一致性损失 (嫌疑预测与最终拒付预测的 logits 方向一致)
    consistency_loss_suspect_new = 1 - F.cosine_similarity(new_user_suspect_prob, final_chargeback_prob, dim=0).mean()
    consistency_loss_suspect_old = 1 - F.cosine_similarity(old_user_suspect_prob, final_chargeback_prob, dim=0).mean()
    L_task_consistency = consistency_loss_suspect_new + consistency_loss_suspect_old


    # 总损失
    total_loss = L_suspect + L_chargeback + cfg.distill_weight * L_distill + cfg.consistency_weight * L_consistency + 0.1 * L_task_consistency # 增加任务一致性损失

    return total_loss, L_suspect, L_chargeback, L_distill, L_consistency, L_task_consistency

# 4. 模型训练函数
def train_model(model, train_loader, optimizer, cfg, teacher_probs=None):
    """
    训练模型一个 epoch。
    Args:
        model (nn.Module): 统一风险模型.
        train_loader (torch.utils.data.DataLoader): 训练数据加载器.
        optimizer (torch.optim.Optimizer): 优化器.
        cfg (Config): 配置对象.
        teacher_probs (dict, optional): 包含四个教师模型预测概率的字典.
            key 为模型名称 (A, B, C, D)，value 为对应的预测概率 DataFrame。
            默认为 None，表示不使用教师模型概率。
    """
    model.train()
    total_loss = 0
    for x, y in train_loader:
        optimizer.zero_grad()
        user_type = x[:, cfg.feature_dim] # 从输入特征中获取用户类型 # (batch_size,)
        # 确保 user_type 的值在有效范围内
        user_type = torch.clamp(user_type, 0, cfg.num_experts - 1)
        new_user_suspect_prob, new_user_chargeback_prob, old_user_suspect_prob, old_user_chargeback_prob, final_chargeback_prob, _ = model(x, user_type)
        loss, L_suspect, L_chargeback, L_distill, L_consistency, L_task_consistency = compute_loss(cfg, new_user_suspect_prob, new_user_chargeback_prob, old_user_suspect_prob, old_user_chargeback_prob, final_chargeback_prob, y, user_type, teacher_probs)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"  Training Loss: {total_loss / len(train_loader):.4f}")

# 5. 模型评估函数
def evaluate_model(model, test_loaders):
    """
    评估模型性能，计算 AUC 和 PR 曲线下的 AUC。
    Args:
        model (nn.Module): 统一风险模型.
        test_loaders (dict): 包含新老用户测试集 DataLoader 的字典.
    Returns:
        dict: 包含新老用户 AUC 和 PR-AUC 的字典.
    """
    model.eval()
    results = {}

    for user_type_str, test_loader in test_loaders.items():
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for x, y in test_loader:
                user_type_batch = x[:, cfg.feature_dim] # 从输入特征中获取用户类型 # (batch_size,)
                # 确保 user_type_batch 的值在有效范围内
                user_type_batch = torch.clamp(user_type_batch, 0, cfg.num_experts - 1).long()
                _, _, _, _, chargeback_prob, _ = model(x, user_type_batch)
                all_preds.extend(chargeback_prob.cpu().numpy())
                all_labels.extend(y.cpu().numpy())

        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)

        # 计算 AUC
        auc_value = roc_auc_score(all_labels, all_preds)

        # 计算 PR-AUC
        precision, recall, _ = precision_recall_curve(all_labels, all_preds)
        pr_auc = auc(recall, precision)

        results[user_type_str] = {
            'auc': auc_value,
            'pr_auc': pr_auc
        }
        print(f"{user_type_str} AUC: {auc_value:.4f}, PR-AUC: {pr_auc:.4f}")
    return results

# 6. 主要训练流程
def main(new_user_data, old_user_data, teacher_probs=None):
    """
    主要训练流程。
    Args:
        new_user_data (pd.DataFrame): 新用户数据，包含特征和标签。
        old_user_data (pd.DataFrame): 老用户数据，包含特征和标签。
        teacher_probs (dict, optional): 包含四个教师模型预测概率的字典.
            key 为模型名称 (A, B, C, D)，value 为对应的预测概率 DataFrame。
            默认为 None，表示不使用教师模型概率。
    Returns:
        tuple: 包含训练集 DataLoader 和测试集 DataLoader 字典的元组。
    """
    # 1. 数据准备
    train_loader, test_loaders = prepare_data(new_user_data, old_user_data, cfg, teacher_probs)

    # 2. 模型初始化
    model = UnifiedRiskModel(cfg)

    # 3. 优化器
    optimizer = torch.optim.Adam(model.parameters(), lr=cfg.learning_rate)

    # 4. 训练循环
    for epoch in range(cfg.num_epochs):
        print(f"Epoch {epoch+1}/{cfg.num_epochs}")
        train_model(model, train_loader, optimizer, cfg, teacher_probs)

        # 5. 评估
        if (epoch + 1) % 10 == 0:
            evaluate_model(model, test_loaders)

    # 在训练结束后调用evaluate_model获取评估结果
    evaluation_results = evaluate_model(model, test_loaders)

    # 6. 保存模型 (可选)
    # torch.save(model.state_dict(), 'unified_risk_model.pth')
    return model, evaluation_results # 返回训练好的模型和测试集loaders

def prepare_data(new_user_data, old_user_data, cfg, teacher_probs=None):
    """
    准备训练和测试数据。
    Args:
        new_user_data (pd.DataFrame): 新用户数据，包含特征和标签。
        old_user_data (pd.DataFrame): 老用户数据，包含特征和标签。
        cfg (Config): 配置对象.
        teacher_probs (dict, optional): 包含四个教师模型预测概率的字典.
            key 为模型名称 (A, B, C, D)，value 为对应的预测概率 DataFrame。
            默认为 None，表示不使用教师模型概率。
    Returns:
        tuple: 包含训练集 DataLoader 和测试集 DataLoader 字典的元组。
    """
    # 1. 数据预处理
    def preprocess_data(df):
        """
        对数据进行预处理，包括填充缺失值和转换为 PyTorch 张量。
        Args:
            df (pd.DataFrame): 输入数据 DataFrame.
        Returns:
            torch.Tensor: 预处理后的特征张量.
            torch.Tensor: 标签张量.
        """
        df = df.fillna(0)  # 简单地填充缺失值
        features = torch.tensor(df.drop('label', axis=1).values, dtype=torch.float32)
        labels = torch.tensor(df['label'].values, dtype=torch.float32)
        return features, labels

    new_user_features, new_user_labels = preprocess_data(new_user_data)
    old_user_features, old_user_labels = preprocess_data(old_user_data)

    # 添加用户类型特征
    new_user_type = torch.zeros(new_user_features.size(0), 1)  # 0 表示新用户
    old_user_type = torch.ones(old_user_features.size(0), 1)   # 1 表示老用户
    new_user_features = torch.cat([new_user_features, new_user_type], dim=1)
    old_user_features = torch.cat([old_user_features, old_user_type], dim=1)

    # 如果提供了教师模型预测概率，则将其添加到特征中
    if teacher_probs and cfg.use_teacher_probs:
        # 将 teacher_probs 转换为 DataFrame 并对齐索引
        teacher_probs_df = pd.concat([teacher_probs['A'], teacher_probs['B'], teacher_probs['C'], teacher_probs['D']], axis=1)
        teacher_probs_df.columns = ['prob_A', 'prob_B',
                                    'prob_C', 'prob_D']
        teacher_probs_df = teacher_probs_df.reset_index(drop=True)

        # 确保 new_user_features 和 old_user_features 的数量与 teacher_probs_df 的行数匹配
        if new_user_features.size(0) + old_user_features.size(0) != teacher_probs_df.shape[0]:
            raise ValueError("Total number of samples in new_user_data and old_user_data must match the number of rows in teacher_probs.")

        # 将 teacher_probs_df 转换为 tensor
        teacher_probs_tensor = torch.tensor(teacher_probs_df.values, dtype=torch.float32)

        # 将 teacher_probs 添加到特征中
        new_user_features = torch.cat([new_user_features, teacher_probs_tensor[:new_user_features.size(0)]], dim=1)
        old_user_features = torch.cat([old_user_features, teacher_probs_tensor[new_user_features.size(0):]], dim=1)

    # 2. 划分训练集和测试集
    new_user_train_features, new_user_test_features, new_user_train_labels, new_user_test_labels = train_test_split(
        new_user_features, new_user_labels, test_size=cfg.test_size, random_state=42
    )
    old_user_train_features, old_user_test_features, old_user_train_labels, old_user_test_labels = train_test_split(
        old_user_features, old_user_labels, test_size=cfg.test_size, random_state=42
    )

    # 3. 创建 DataLoader
    train_features = torch.cat([new_user_train_features, old_user_train_features], dim=0)
    train_labels = torch.cat([new_user_train_labels, old_user_train_labels], dim=0)
    train_dataset = TensorDataset(train_features, train_labels)
    train_loader = DataLoader(train_dataset, batch_size=cfg.batch_size, shuffle=True)

    # 创建测试集 DataLoader 字典
    test_loaders = {
        'new_user': DataLoader(TensorDataset(new_user_test_features, new_user_test_labels), batch_size=cfg.batch_size, shuffle=False),
        'old_user': DataLoader(TensorDataset(old_user_test_features, old_user_test_labels), batch_size=cfg.batch_size, shuffle=False),
    }

    return train_loader, test_loaders

if __name__ == '__main__':
    # 模拟数据生成 (使用 Pandas DataFrame)
    def generate_synthetic_data(num_samples, num_features):
        """
        生成模拟数据。
        Args:
            num_samples (int): 样本数量.
            num_features (int): 特征数量.
        Returns:
        pd.DataFrame: 包含模拟数据的 DataFrame.
        """
        data = np.random.rand(num_samples, num_features)
        labels = np.random.randint(0, 2, num_samples)  # 随机生成 0 和 1 的标签
        df = pd.DataFrame(data, columns=[f'feature_{i}' for i in range(num_features)])
        df['label'] = labels
        return df

    # 生成新用户和老用户数据
    num_features = 100
    new_user_data = generate_synthetic_data(1000, num_features)
    old_user_data = generate_synthetic_data(1500, num_features)

    # 模拟教师模型预测概率 (DataFrame)
    teacher_probs = {
        'A': pd.DataFrame(np.random.rand(2500, 1), columns=['prob_A']),
        'B': pd.DataFrame(np.random.rand(2500, 1), columns=['prob_B']),
        'C': pd.DataFrame(np.random.rand(2500, 1), columns=['prob_C']),
        'D': pd.DataFrame(np.random.rand(2500, 1), columns=['prob_D']),
    }

    # 运行主要训练流程
    trained_model, evaluation_results = main(new_user_data, old_user_data, teacher_probs)

    # 打印测试结果
    print("Final Test Results:")
    for user_type, metrics in evaluation_results.items():
        print(type(user_type))
        print(f"{user_type.capitalize()} - AUC: {metrics['auc']:.4f}, PR-AUC: {metrics['pr_auc']:.4f}")


Epoch 1/50
  Training Loss: 1.1473
Epoch 2/50
  Training Loss: 1.0969
Epoch 3/50
  Training Loss: 1.0795
Epoch 4/50
  Training Loss: 1.0660
Epoch 5/50
  Training Loss: 1.0605
Epoch 6/50
  Training Loss: 1.0546
Epoch 7/50
  Training Loss: 1.0518
Epoch 8/50
  Training Loss: 1.0497
Epoch 9/50
  Training Loss: 1.0477
Epoch 10/50
  Training Loss: 1.0473
new_user AUC: 0.5124, PR-AUC: 0.5328
old_user AUC: 0.4760, PR-AUC: 0.4704
Epoch 11/50
  Training Loss: 1.0472
Epoch 12/50
  Training Loss: 1.0432
Epoch 13/50
  Training Loss: 1.0413
Epoch 14/50
  Training Loss: 1.0387
Epoch 15/50
  Training Loss: 1.0359
Epoch 16/50
  Training Loss: 1.0345
Epoch 17/50
  Training Loss: 1.0329
Epoch 18/50
  Training Loss: 1.0309
Epoch 19/50
  Training Loss: 1.0318
Epoch 20/50
  Training Loss: 1.0315
new_user AUC: 0.4978, PR-AUC: 0.5214
old_user AUC: 0.4743, PR-AUC: 0.4700
Epoch 21/50
  Training Loss: 1.0313
Epoch 22/50
  Training Loss: 1.0308
Epoch 23/50
  Training Loss: 1.0304
Epoch 24/50
  Training Loss: 1.03


### 统一模型构建方案

---

#### 一、架构设计：层次化多专家蒸馏网络
**1. 输入层特征工程**
- **基础特征**：合并新老用户特征，标准化处理：
  - 用户类型（新/老）作为显式二值特征
  - 动态字段映射：对存在差异的特征字段（如新用户缺少年度消费次数），使用零值填充并添加缺失标识
- **教师信号特征**：将A/B/C/D模型的预测概率作为辅助输入特征

**2. 核心网络结构**
```mermaid

graph TD
    A[输入层] --> B[双向门控共享层\nBi-Gated Dense Block]
    B --> C[新用户专家通道\nGBDT特征蒸馏]
    B --> D[老用户专家通道\nGBDT特征蒸馏]
    C --> E[嫌疑预测任务头\nSigmoid输出]
    D --> F[嫌疑预测任务头\nSigmoid输出]
    E --> G[动态权重融合层\nAttention]
    F --> G
    G --> H[最终拒付预测\nSigmoid输出]

```


---

#### 二、关键技术实现

**1. 双向门控共享层**
- **门控机制**：根据用户类型动态激活通道
  ```python
  # 伪代码示例
  gate_new = σ(W_g * concat(输入特征, 用户类型标识))
  shared_feature = gate_new * Dense_New(输入特征) + (1-gate_new) * Dense_Old(输入特征)
  ```

- **蒸馏约束**：添加KL散度损失，强制专家通道的 logits 匹配教师模型的 logits

**2. 专家通道设计**
- **新用户通道**：
  - 输入：共享层输出 + 模型 A/B 预测概率
  - 子结构：两个全连接网络，分别进行嫌疑预测和拒付预测
  - 损失函数：交叉熵损失 （嫌疑预测）和 Focal Loss （拒付预测）
- **老用户通道**：
  - 输入：共享层输出 + 模型 C/D 预测概率
  - 子结构：两个全连接网络，分别进行嫌疑预测和拒付预测
  - 损失函数：Focal Loss（用于嫌疑预测和拒付预测）

**3. 注意力融合层**
- **跨通道注意力**：
  ```python
  attention_score = Softmax(MLP(concat(新用户嫌疑预测, 新用户拒付预测, 老用户嫌疑预测, 老用户拒付预测)))
  fused_feature = attention_score[0] * 新用户预测 + attention_score[1] * 老用户预测
  ```

- **任务一致性约束**：添加余弦相似度损失项，确保新老用户的嫌疑预测与最终拒付预测的 logits 方向一致

---

#### 三、训练策略设计


**1. 训练流程**

端到端联合训练


**2. 损失函数设计**

$$L_{total} = \alpha L_{suspect} + \beta L_{chargeback} + \gamma L_{distill} + \lambda L_{consistency} + \theta L_{task\_consistency}$$

- $L_{suspect}$：嫌疑预测损失（新用户 BCE / 老用户 Focal Loss）
- $L_{chargeback}$：拒付预测损失（Focal Loss）
- $L_{distill}$：教师模型 KL 散度损失（温度缩放 T=3）
- $L_{consistency}$：新老用户各自的嫌疑预测和拒付预测之间的一致性损失（MSE）
- $L_{task_consistency}$：任务一致性损失（新老用户嫌疑预测与最终拒付预测 logits 的余弦相似度损失）


**3. 动态权重调整**

使用固定的超参数：$\alpha$、$\beta$、$\gamma$、$\lambda$、$\theta$

---

#### 四、数据流与特征增强

**1. 影子样本生成**

（代码中未使用）

**2. 时间序列增强**

（代码中未使用）

---

#### 五、部署与监控方案

**1. 渐进式上线策略**

（代码中未使用）

**2. 实时监控看板**

（代码中未使用）

---

#### 六、预期效果与优势
1. **效果提升**：
   - 新用户风险识别能力提升
   - 老用户拒付召回率提升
   - 模型推理效率提升

2. **技术优势**：
   - 通过门控机制实现新老用户的动态区分和特征共享
   - 注意力融合层实现新老用户预测结果的动态加权融合
   - 知识蒸馏从教师模型迁移知识，提升学生模型性能
   - 一致性损失约束模型学习到更加一致的表征
   - 任务一致性损失约束模型学习到相关性更高的特征表示

该方案通过结合门控机制、注意力融合和知识蒸馏等技术，利用新老用户数据和教师模型知识，增强模型的风险识别能力和泛化性。


# 统一风险模型改进方案

在现有模型基础上，通过以下维度提升模型性能和鲁棒性：

---

## 一、训练优化

### 1. 三阶段训练流程
| 阶段      | 训练目标                          | 数据使用策略                     | 关键操作                                                                 |
|-----------|-----------------------------------|----------------------------------|--------------------------------------------------------------------------|
| 预训练    | 专家通道独立初始化                | 新/老用户数据隔离                | - 分别训练新老用户专家通道<br>- 使用简单损失函数                         |
| 联合训练  | 共享层+任务头端到端优化           | 全量数据混合，动态批次平衡       | - 混合新老用户数据<br>- 引入动态批次平衡策略<br>- 完整损失函数端到端训练 |
| 微调      | 注意力融合层+最终层调优           | 添加拒绝推断扩展数据             | - 重点调整融合层参数<br>- 引入拒绝推断数据<br>- 小学习率精细调参         |

### 2. 动态权重调整
**自适应机制：**
```python
alpha = 1 - (当前epoch / max_epoch)  # 逐步降低嫌疑预测权重
beta = 0.5 * (当前拒付召回率 / 目标召回率)
```

**参数调整策略：**
- 早期阶段：提高蒸馏损失权重（gamma↑）
- 后期阶段：增强一致性损失权重（lambda↑）
- 召回率不足时：提升拒付损失权重（beta↑）

---

## 二、数据增强方案

### 1. 影子样本生成
**实现方法：**
```text
WGAN-GP生成器 → 教师模型标注 → 加入训练集
```
**优势：**
- 生成新老用户边界样本
- 利用教师模型知识自动标注
- 提升模型鲁棒性

### 2. 时序特征增强（老用户专属）
**实施流程：**
```
交易时序数据 → Transformer编码 → 特征拼接
```
**特征维度：**
- 30天交易频率
- 金额波动序列
- 历史拒付次数

**技术优势：**
- 捕捉长期行为模式
- 增强静态特征表达能力

---

## 三、部署与监控

### 1. 渐进式上线策略
| 用户类型 | 阶段1权重          | 阶段2权重          | 阶段3权重          |
|----------|--------------------|--------------------|--------------------|
| 新用户   | 模型A:70%          | 统一模型:100%      | -                  |
| 老用户   | 模型D:50%          | 统一模型:80%       | 统一模型:100%      |

**演进特点：**
- 分阶段流量切换
- 异常快速回滚能力
- 新旧模型无缝过渡

### 2. 实时监控体系
**核心指标：**
```text
新用户误杀率（A/B测试）
老用户召回率提升幅度
跨国特征PSI指数（阈值<0.15）
```

**熔断机制：**
```python
if 连续5批次预测方差 > 阈值:
    自动切换至教师模型组合
```

**监控能力：**
- 实时特征漂移检测
- 多维度性能对比
- 自动化异常处理

---

## 四、预期收益
1. **性能提升**：通过时序特征增强预计提升老用户召回率3-5%
2. **稳定性增强**：影子样本机制可使模型对抗性样本误判率降低40%+
3. **部署安全**：渐进式上线策略将新模型风险暴露降低70%
4. **可解释性**：动态注意力权重提供风险决策依据
